In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
from testing_freq import *
from utils import *

%matplotlib inline

### Load Copenhagen Network Data

In [3]:
bt = pd.read_csv('bt_data_clean.csv', header = 0, names = ['timestamp','user_a', 'user_b', 'rssi'])
bt.head()

,timestamp,user_a,user_b,rssi
0,0,20,19,-73
1,0,46,18,-86
2,0,48,47,-62
3,0,68,43,-87
4,0,89,63,-96


### Run Simulations

#### Construct adjacency matrices.

In [4]:
time_step = 86400
start = time.time()
adj_mats, df = construct_adj_mat(bt, time_step = time_step, data_loops = 3, dist_thres = -75)
end = time.time()
end - start

232.69882225990295

In [5]:
adj_mats.shape

(112, 692, 692)

#### Set up the simulation parameters.

In [6]:
myseed = 12

disease_params = dict()
disease_params['ext_inf'] = 0.02
disease_params['asymp'] = 0.3
disease_params['init_status'] = gen_init_status(n_nodes = adj_mats.shape[1], asymp = disease_params['asymp'], seed = myseed)
disease_params['beta'] = gen_trans_prob_NPI(n_nodes = adj_mats.shape[1], base_beta = 0.007, p_sd = 0.2, p_fm = 0.2, seed = myseed)
disease_params['sigma_a'] = 0.3
disease_params['sigma_s'] = 0.3
disease_params['gamma_a'] = 1/7
disease_params['gamma_s'] = 1/10
disease_params['n_time_day'] = 1

test_params = dict()
test_params['test_freq'] = 3
test_params['spec'] = 0.99
test_params['symp_test_delay'] = gen_symp_test_delay(n_nodes = adj_mats.shape[1], univ_delay = 3)
test_params['nc_schd'] = 0.05
test_params['nc_symp'] = 0.1
test_params['time_dep'] = True
test_params['time_dep_type'] = 'W'
test_params['false_symp'] = 0.05

quar_params = dict()
quar_params['quar_delay'] = gen_quar_delay(n_nodes = adj_mats.shape[1], univ_delay = 1)
quar_params['quar_comp'] = gen_quar_comp(n_nodes = adj_mats.shape[1], seed = myseed)
quar_params['quar_len'] = 10

In [7]:
# Instantiate the simulation class
testin = TestFreq(adj_mats, disease_params, test_params, quar_params)
testin

#### Run simulation without testing or isolation.

In [8]:
(s_nodes_byt, e_nodes_byt, ia_nodes_byt, is_nodes_byt, r_nodes_byt) = testin.sim_spread(seed = myseed)

In [9]:
all_inf = list(set(np.concatenate(ia_nodes_byt + is_nodes_byt).ravel()))
len(all_inf)

661

#### Run simulation with testing and isolation.

In [10]:
start = time.time()
(ia_nodes_byt, is_nodes_byt, test_pos_schd_byt, test_pos_symp_byt, q_schd_byt, q_symp_byt) = testin.sim_spread_test(seed = myseed)
end = time.time()
end - start

4.8690245151519775

In [11]:
# Extract the total number infected
all_inf = list(set(np.concatenate(ia_nodes_byt + is_nodes_byt).ravel()))
len(all_inf)

631